In [2]:
#tkinter is a python GUI library being used as the framework
import tkinter
import random
import math
from tkinter import messagebox

In [3]:
#The below class moves the point when it is dragged using a mouse
class task1:
    def __init__(self, window, x, y):
        self.window = window
        self.oval = self.window.create_oval(x, y, x+10, y+10, fill="black", tags="point")
        self.window.tag_bind(self.oval, '<Button1-Motion>', self.on_drag)

    def on_drag(self, event):
        x, y = event.x, event.y
        self.window.coords(self.oval, x, y, x+10, y+10)

def deleteall():
    window.delete("all") 
def deleteselected(event):
    item = window.find_closest(event.x, event.y)
    if item:
        window.delete(item)

def generatepoints(): 
    for _ in range(5):
        x = random.randint(0, 485) 
        y = random.randint(0, 485) 
        task1(window, x, y)

def clickpoint(event):
    item = window.find_overlapping(event.x, event.y, event.x, event.y)
    if not item:
        x, y = event.x, event.y
        task1(window, x, y)

In [14]:
#convex hull using gift wrapping

def gift_wrapping_convex_hull(points):
    if len(points) < 3:
        return []

    hull = []
    start_point = max(points, key=lambda point: point[0])  # Choose the point with highest x-coordinate
    hull.append(start_point)
    current_point = start_point

    while True:
        endpoint = points[0]
        for next_point in points[1:]:
            if current_point == endpoint or \
                    (current_point[0] == endpoint[0] and current_point[1] > endpoint[1]) or \
                    (current_point[0] == next_point[0] and current_point[1] > next_point[1]) or \
                    (endpoint[0] - current_point[0]) * (next_point[1] - current_point[1]) - \
                    (next_point[0] - current_point[0]) * (endpoint[1] - current_point[1]) < 0:
                endpoint = next_point

        if endpoint == start_point:
            break

        hull.append(endpoint)
        current_point = endpoint

    return hull

def draw_convex_hull(algorithm):
    global hull_points, graham, gift
    points = []
    for item in window.find_withtag("point"):
        coords = window.coords(item)
        points.append((coords[0] + 2, coords[1] + 2))
    
    window.delete("convex_hull")

    if algorithm=="graham":
        graham=graham_scan(points)
        graham_coords = [coord for point in graham for coord in point]
        window.create_polygon(graham_coords, outline='blue', fill='', width=2, tags="convex_hull")
        messagebox.showinfo("Convex Hull", "Convex Hull: Graham Scan")

    if algorithm=="gift":
        gift=gift_wrapping_convex_hull(points)
        hull_coords = [coord for point in gift for coord in point]
        window.create_polygon(hull_coords, outline='blue', fill='', width=2, tags="convex_hull")
        messagebox.showinfo("Convex Hull", "Convex Hull: Gift Wrapping")
    
def draw_graham_hull():
    global graham_hull
    points = []
    for item in window.find_withtag("point"):
        coords = window.coords(item)
        points.append((coords[0] + 2, coords[1] + 2))

    graham_hull = gift_wrapping_convex_hull(points)
    window.delete("convex_hull")

    if graham_hull:
        graham_coords = [coord for point in graham_hull for coord in point]
        window.create_polygon(graham_coords, outline='blue', fill='', width=2, tags="convex_hull")

        # Triangulate the convex hull using a sweep line algorithm
        triangulate_convex_hull(graham_hull)



In [13]:
#convex hull using graham scan

def graham_scan(points):
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0  # Collinear
        return 1 if val > 0 else 2  # Clockwise or counterclockwise

    n = len(points)
    if n < 3:
        return points

    hull = []

    # Find the point with the lowest y-coordinate (and leftmost in case of ties) as the pivot
    pivot = min(points, key=lambda point: (point[1], point[0]))
    hull.append(pivot)

    sorted_points = sorted(points, key=lambda point: (math.atan2(point[1]-pivot[1], point[0]-pivot[0]), point))

    for point in sorted_points:
        while len(hull) > 1 and orientation(hull[-2], hull[-1], point) != 2:
            hull.pop()
        hull.append(point)

    return hull

In [6]:
#there is a problem when the convex hull is concave
#def triangulate_convex_hull(convex_hull):
    n = len(convex_hull)
    if n < 3:
        return 


    convex_hull.sort()

    stack = [convex_hull[0], convex_hull[1]]

    for i in range(2, n):
        vi = convex_hull[i]
        while len(stack) > 1:
            vj = stack[-1]
            vk = stack[-2]
            if (vi[0] - vk[0]) * (vj[1] - vk[1]) - (vi[1] - vk[1]) * (vj[0] - vk[0]) > 0:

                for j in range(len(stack) - 1):
                    window.create_polygon([vi, stack[j], stack[j + 1]], outline='green', fill='', width=2, tags="triangles")
                stack = stack[:-1] 
            else:
                stack.pop()

        stack.append(vi)

   
    vtop = stack[0]
    for i in range(1, len(stack) - 1):
        window.create_polygon([vtop, stack[i], stack[i + 1]], outline='green', fill='', width=2, tags="triangles")


In [12]:
def triangulate_convex_hull(convex_hull):
    n = len(convex_hull)
    if n < 3:
        return  # Convex hull has less than 3 vertices, so no triangles

    triangles = []  # List to store triangles

    # Sort vertices lexicographically (in this case, by x-coordinate)
    convex_hull.sort()

    # Add the first two vertices to the stack
    stack = [convex_hull[0], convex_hull[1]]

    for i in range(2, n):
        vi = convex_hull[i]
        # Check if vi forms a convex vertex or is part of a concave structure
        while len(stack) > 1:
            vj = stack[-1]
            vk = stack[-2]

            if is_convex_vertex(vk, vj, vi):
                triangles.append([vk, vj, vi])
                stack.pop()
            else:
                break

        stack.append(vi)

    # Triangulate the remaining vertices
    vtop = stack[0]
    for i in range(1, len(stack) - 1):
        triangles.append([vtop, stack[i], stack[i + 1]])

    # Draw the triangles
    for triangle in triangles:
        window.create_polygon(triangle, outline='green', fill='', width=2, tags="triangles")

def is_convex_vertex(vk, vj, vi):
    # Check if vi forms a convex vertex (vk, vj, vi are in counterclockwise order)
    return (vi[0] - vk[0]) * (vj[1] - vk[1]) - (vi[1] - vk[1]) * (vj[0] - vk[0]) > 0


In [15]:
#The user interface
a1 = tkinter.Tk()
window = tkinter.Canvas(a1, width=500, height=500)
a1.title("Graphics Project")
window.pack()
window.bind("<Button-1>", clickpoint)
window.bind("<Button-3>", deleteselected)
button = tkinter.Button(a1, text="Generate Random Points", command=generatepoints)
button.pack(pady=5)
button = tkinter.Button(a1, text="Delete Points", command=deleteall)
button.pack(pady=5)
convex_hull_button = tkinter.Button(a1, text="Compute Convex Hull", command=lambda: draw_convex_hull("gift"))
convex_hull_button.pack(pady=10)
graham_button = tkinter.Button(a1, text="Compute Convex hull using graham scan", command=lambda: draw_convex_hull("graham"))
graham_button.pack(pady=10)
graham_button = tkinter.Button(a1, text="Triangulation of convex hull", command=draw_graham_hull)
graham_button.pack(pady=10)
a1.mainloop()